### Step 2: Attribute Tiff Data to HydroBasins Level 12

Attributing information from Tiff files to HydroBASINS Level 12

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import json
from rasterstats import zonal_stats
from timeit import default_timer as timer

from utils import file_management as f_mng
from utils import attribution as attr

First we will import tif file processing information and HydroBASIN data that we stored in step 1 (see step1_data_management.ipynb).  

We then determine bounds for each HydroBASIN.  In some cases TIF files do not have global coverage.  Instead of running the entire (CPU intense) zonal statistics process on each TIF we run a quick box intersection algorithm to test which TIF files likely have relevant data for each basin and only run the rest of the attribution process on these files. 

Note there is a lot of data being loaded, this cell may take over a minute to load.

In [2]:
#Import file processing information
with open("data/var/tif_vars_file_info.json", "r") as all_file_info:
    json_file_info = json.load(all_file_info)
    
#Read in list of HYBAS_IDs (see step1_data_management.ipynb).  Provides list of IDs to process.
hybas_id_list=f_mng.read_pkl_df('data/basins_lvl12.txt')

#Read in pickled level 12 HydroBASINS (see step1_data_management.ipynb)
gdf = f_mng.read_pkl_gdf()

This example is only running on a few basins as an example. When running all basins this can take hours using one processor.  See a multiprocessing python script that was used to run attribution of all tif files for all basins (XXXXXX.py).


In [3]:
import warnings
warnings.filterwarnings(action='once')

all_stats = []

#run attribution on one basin, on subset of basins 0:3
for basin in hybas_id_list[0:3]:
    
    #Select row from geodataframe with basin information, including bounding box info
    basin_info_gdf = gdf.loc[gdf['HYBAS_ID']==int(basin)]
    pfaf_12 = basin_info_gdf.iloc[0]['PFAF_ID']
    sub_area = float((basin_info_gdf.iloc[0])['SUB_AREA'])
    
    bbox_gdf = basin_info_gdf.bounds

    #Initiate basin stat object, this will help manage stats as created for the basin
    basin_info = attr.Stats(basin, pfaf_12, sub_area)

    #Add bounding box of basin to object
    basin_info.bounds((bbox_gdf['minx']),(bbox_gdf['maxx']),(bbox_gdf['miny']), (bbox_gdf['maxy']))
    #basin_info.bounds((basin_info_gdf['minx']),(basin_info_gdf['maxx']),(basin_info_gdf['miny']), (basin_info_gdf['maxy']))
    
    # loop through each record (representing a file in the folder containing variable data) in json_file_info 
    for file in json_file_info:
            
        var_bounds = file['bounds']

        #evaluate if data in file spatialy overlaps spatial object using bounds, process if overlap/intersection occurs
        basin_info.evaluate_intersection(var_bounds)
        #If needed run zonal stats
        basin_info.run_zonal_stats(basin_info_gdf, file)
    
    #Add stats to a common list
    all_stats = basin_info.add_to_all_stats(all_stats)
    basin_info = None
    

/home/dwief/anaconda3/lib/python3.7/site-packages/rasterstats/io.py:300: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/dwief/anaconda3/lib/python3.7/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


First record in all_stats.  Note that we are capturing more data than we likely need for our end product.  These information can help us test results and help others track where data are coming from.

Each Record contains the following:
 * id: hybas_id from hydrobasins (level 12)
 * pfaf_12: pfaf value from hydrobasins
 * suite of summaries: 
         * label_stat0:value0, label_stat1:value1... label_statx:valuex
         * src_file: tif files used in summary and how they evaluated in the bounds_eval method

In [4]:
all_stats[0]

{'h2o_depletion': {'h2o_depletion_1.0': 2,
  'h2o_depletion_min': 1.0,
  'h2o_depletion_max': 1.0,
  'h2o_depletion_majority': 1.0,
  'src_file': [{'file_name': 'WaterDepletionCat_WG3.tif', 'bounds_eval': 1}]},
 'pasture2000': {'pasture2000_mean': 0.8507096171379089,
  'pasture2000_count': 2,
  'pasture2000_nodata': 0.0,
  'src_file': [{'file_name': 'Pasture2000_5m_update_nodata999.tif',
    'bounds_eval': 1}]},
 'cropland2000': {'cropland2000_mean': 0.14929038286209106,
  'cropland2000_count': 2,
  'cropland2000_nodata': 0.0,
  'src_file': [{'file_name': 'Cropland2000_5m_update_nodata999.tif',
    'bounds_eval': 1}]},
 'gpw2015_pop_den': {'gpw2015_pop_den_mean': 18.127021703827246,
  'gpw2015_pop_den_count': 178,
  'gpw2015_pop_den_nodata': 0.0,
  'src_file': [{'file_name': 'gpw_v4_population_density_rev11_2015_30_sec.tif',
    'bounds_eval': 1}]},
 'lc2015_bare_cov': {'lc2015_bare_cov_mean': 0.0,
  'lc2015_bare_cov_count': 12881,
  'lc2015_bare_cov_nodata': 0.0,
  'src_file': [{'file

In [5]:
#Export information to json file

#with open('ouput/tif_att_stats.json', 'w') as outfile:
#    json.dump(all_stats, outfile)